In [1]:
!nvidia-smi

Fri May 14 18:15:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:05.0 Off |                  Off |
| N/A   27C    P0    54W / 250W |      0MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

apex is not installed
apex is not installed
apex is not installed
apex is not installed


In [3]:
# config file 들고오기
cfg = Config.fromfile('./configs/swin/casmaskrcnn_swin_base_p4_win7_adamw_Crunner_trash.py')

cfg.seed = 42
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/casmaskrcnn_swin_base_p4_win7_adamw_Crunner_trash_1'

#cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [4]:
group_name = 'cascade_swinbase'; project_name = 'kw_ojd_p3_2'; run_name = "1_autoaug_anchoradded_rpn"
config_list = {
    'epoch' : cfg.runner.max_epochs,
    'batch_size' :  cfg.data.samples_per_gpu,
    'optimizer' : cfg.optimizer,
    'optimizer_config' : cfg.optimizer_config,
    'lr_config' : cfg.lr_config
}
cfg.log_config.hooks[1].init_kwargs['group'] = group_name # group name(option)
cfg.log_config.hooks[1].init_kwargs['name'] = run_name # run name
cfg.log_config.hooks[1].init_kwargs['config'] = config_list # config

In [5]:
model = build_detector(cfg.model)

2021-05-14 18:15:55,327 - mmdet - INFO - load model from: /opt/ml/code/pretrained_pth/swin_base_patch4_window7_224.pth
2021-05-14 18:15:56,389 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



In [6]:
from mmcv.runner import load_checkpoint
checkpoint = load_checkpoint(model, "../pretrained_pth/cascade_mask_rcnn_swin_base_patch4_window7.pth", map_location='cuda:0')

Use load_from_local loader
The model and loaded state dict do not match exactly

size mismatch for rpn_head.rpn_cls.weight: copying a param with shape torch.Size([3, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([12, 256, 1, 1]).
size mismatch for rpn_head.rpn_cls.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for rpn_head.rpn_reg.weight: copying a param with shape torch.Size([12, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([48, 256, 1, 1]).
size mismatch for rpn_head.rpn_reg.bias: copying a param with shape torch.Size([12]) from checkpoint, the shape in current model is torch.Size([48]).
size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) fr

In [7]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.51s)
creating index...
index created!


In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-14 18:16:17,382 - mmdet - INFO - Start running, host: root@45b74c7c03c9, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/casmaskrcnn_swin_base_p4_win7_adamw_Crunner_trash_1
2021-05-14 18:16:17,383 - mmdet - INFO - workflow: [('train', 1)], max: 20 epochs


Done (t=0.83s)
creating index...
index created!


wandb: Currently logged in as: pstage12 (use `wandb login --relogin` to force relogin)


2021-05-14 18:16:40,278 - mmdet - INFO - Epoch [1][10/524]	lr: 1.898e-06, eta: 5:22:49, time: 1.850, data_time: 0.264, memory: 14431, loss_rpn_cls: 0.8939, loss_rpn_bbox: 1.5167, s0.loss_cls: 2.2178, s0.acc: 36.2891, s0.loss_bbox: 0.0544, s0.loss_mask: 2.3031, s1.loss_cls: 1.5144, s1.acc: 0.0469, s1.loss_bbox: 0.0060, s1.loss_mask: 1.2513, s2.loss_cls: 0.6704, s2.acc: 0.0938, s2.loss_bbox: 0.0008, s2.loss_mask: 0.2925, loss: 10.7213
2021-05-14 18:16:56,156 - mmdet - INFO - Epoch [1][20/524]	lr: 3.896e-06, eta: 4:59:39, time: 1.588, data_time: 0.022, memory: 14431, loss_rpn_cls: 0.8915, loss_rpn_bbox: 1.4425, s0.loss_cls: 2.1709, s0.acc: 54.4180, s0.loss_bbox: 0.0507, s0.loss_mask: 2.1224, s1.loss_cls: 1.5029, s1.acc: 0.0234, s1.loss_bbox: 0.0067, s1.loss_mask: 1.1514, s2.loss_cls: 0.6665, s2.acc: 0.0664, s2.loss_bbox: 0.0007, s2.loss_mask: 0.3101, loss: 10.3163
2021-05-14 18:17:12,031 - mmdet - INFO - Epoch [1][30/524]	lr: 5.894e-06, eta: 4:51:44, time: 1.587, data_time: 0.019, memory: